In [ ]:
import os, sys
sys.path.append(os.path.abspath("../"))

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from config import defaults
from workplaces.workplace import WorkPlace

from calculators.reg_tr_calculator import TR_calculator as RegTrCalc
from calculators.reg_base_flow_matrix_calculator import RegBaseFlowCalc
from calculators.reg_dm_calculator import RegDMCalc


In [ ]:
# Рабочая область
wp = WorkPlace(name='rounet_loko',
               config=defaults,
               db_type='mongo')
# Дата провайдер
dp = wp.data_provider

# wp.db.drop_table('test')

### Генерация и сохранение ТР

In [ ]:
# municipalities_df = dp.get_dataset(dataset_name='mun_soc')

# tr_calc = RegTrCalc(municipalities_df)
# tr_calc.generate_tr()

# dp.save_dataset(dataset=tr_calc.trs,
#                 collection_name='trs',
#                 version_name='base')

# trs = dp.get_dataset(dataset_name='trs',
#                      version_name='base')

### Генерация базовой матрицы потоков

In [ ]:
trs = dp.get_dataset(dataset_name='trs',
                     version_name='base')
flows_base = dp.get_dataset(dataset_name='flows')
plos_express_info = dp.get_dataset(dataset_name='plos_express_info')

regbf_calc = RegBaseFlowCalc(trs=trs,
                             flows_base=flows_base,
                             plos_express_info=plos_express_info)
regbf_calc.generate_base_matrix()

dp.save_dataset(dataset=regbf_calc.save_dict,
                collection_name='flow_base_matrix',
                version_name='base')

fbm = dp.get_dataset(dataset_name='flow_base_matrix',
                version_name='base')

### Генерация матрицы расстояний

In [ ]:
# trs = dp.get_dataset(dataset_name='trs',
#                      version_name='base')

# reg_dm_calc = RegDMCalc(trs=trs,
#                         calc_type='by_ort')
# reg_dm_calc.generate_dm()

# dp.save_dataset(dataset=reg_dm_calc.save_dict,
#                 collection_name='dist_matrix',
#                 version_name='base')

# dm = dp.get_dataset(dataset_name='dist_matrix',
#                 version_name='base')

### Расчет матрицы корреспонденций на прогнозируемый год

In [ ]:
trs = dp.get_dataset(dataset_name='trs',
                     version_name='base')

fbm = dp.get_dataset(dataset_name='flow_base_matrix',
                version_name='base')

dm = dp.get_dataset(dataset_name='dist_matrix',
                version_name='base')

In [ ]:
fbm.sum()

#### Определение потенциалов фомирования и притяжения в базисном году

In [ ]:
base_attractions = (np.sum(fbm, axis = 1) + np.sum(fbm, axis = 0))/2
base_generations = attractions

#### Определение потенциалов формирования и притяжения в прогнозном году

In [ ]:
def get_indicies(trs, soc_type, base_year,forecast_year, default_val=0.001):
    contain = []
    for key, val in trs.items():
        base_val = val[soc_type][str(base_year)]
        forecast_val = val[soc_type][str(forecast_year)]
        
        if not base_val:
            base_val = default_val
        if not forecast_val:
            forecast_val = default_val
        index = forecast_val/base_val
        contain.append(index)
    # Дли иностранного ТР
    contain.append(np.mean(np.array(contain)))   
    res = np.array(contain)
    return res

def calc_generation_indicies(pop_indicies, grp_indicies, employ_indicies, egg):
    gt = np.array([pop_indicies[i]*
          (1+egg[i]*(grp_indicies[i]-1))
          for i in range(len(pop_indicies))])
    gb = np.array([employ_indicies[i]*
          (1+egg[i]*(grp_indicies[i]-1))
          for i in range(len(pop_indicies))])
    go = np.array([pop_indicies[i]*
          (1+egg[i]*(grp_indicies[i]-1))
          for i in range(len(pop_indicies))])
    
    res = (gt + gb + go)/3
    return res

def calc_attraction_indicies(pop_indicies, grp_indicies, employ_indicies, bed_indicies, ega):
    at = np.array([bed_indicies[i]*
          (1+ega[i]*(grp_indicies[i]-1))
          for i in range(len(pop_indicies))])
    ab = np.array([employ_indicies[i]*
          (1+ega[i]*(grp_indicies[i]-1))
          for i in range(len(pop_indicies))])
    ao = np.array([pop_indicies[i]*
          (1+ega[i]*(grp_indicies[i]-1))
          for i in range(len(pop_indicies))])
    
    res = (at + ab + ao)/3
    return res

In [ ]:
base_attractions = (np.sum(fbm, axis = 1) + np.sum(fbm, axis = 0))/2
base_generations = attractions

base_year = 2019
forecast_year = 2035

default_el_grp_val = 0.3

pop_indicies = get_indicies(trs, 'POP', base_year,forecast_year)
employ_indicies = get_indicies(trs, 'EMPLOY', base_year,forecast_year)
grp_indicies = get_indicies(trs, 'GRP', base_year,forecast_year)
bed_indicies = get_indicies(trs, 'BED', base_year,forecast_year)


# elasticities_grp_attraction
ega = np.array([default_el_grp_val for i in range(len(fbm))])
# elasticities_grp_generation
egg = np.array([default_el_grp_val for i in range(len(fbm))])

generation_indicies = calc_generation_indicies(pop_indicies, grp_indicies, employ_indicies, egg)
attraction_indicies = calc_attraction_indicies(pop_indicies, grp_indicies, employ_indicies, bed_indicies, ega)

forecast_generations = base_generations*generation_indicies
forecast_attractions = base_attractions*attraction_indicies

(sum(forecast_generations)+sum(forecast_attractions))/(2*sum(base_attractions))